In [0]:
import pandas as pd

Dataset is taken from 
https://www.kaggle.com/blastchar/telco-customer-churn#WA_Fn-UseC_-Telco-Customer-Churn.csv

In [720]:
file_url = "/content/WA_Fn-UseC_-Telco-Customer-Churn.csv"

data = pd.read_csv(file_url)
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


Remove the ID since we do not need the unique id for the model

In [721]:
data = data.drop("customerID",axis=1)
data.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


Get the feature and label to train our model

In [0]:
features = data.drop("Churn",axis=1)

label = data[["Churn"]]

In [0]:
import numpy as np

We wanted to find the unique value of category feature

In [724]:
def print_unique(df:pd.DataFrame,column_name:str):
  print(column_name)
  print(np.unique(df[column_name]))

print_unique(features,"gender")
print_unique(features,"SeniorCitizen")
print_unique(features,"Partner")
print_unique(features,"Dependents")
print_unique(features,"PhoneService")
print_unique(features,"MultipleLines")
print_unique(features,"InternetService")
print_unique(features,"OnlineSecurity")
print_unique(features,"OnlineBackup")
print_unique(features,"DeviceProtection")
print_unique(features,"TechSupport")
print_unique(features,"StreamingTV")
print_unique(features,"StreamingMovies")
print_unique(features,"Contract")
print_unique(features,"PaperlessBilling")
print_unique(features,"PaymentMethod")

gender
['Female' 'Male']
SeniorCitizen
[0 1]
Partner
['No' 'Yes']
Dependents
['No' 'Yes']
PhoneService
['No' 'Yes']
MultipleLines
['No' 'No phone service' 'Yes']
InternetService
['DSL' 'Fiber optic' 'No']
OnlineSecurity
['No' 'No internet service' 'Yes']
OnlineBackup
['No' 'No internet service' 'Yes']
DeviceProtection
['No' 'No internet service' 'Yes']
TechSupport
['No' 'No internet service' 'Yes']
StreamingTV
['No' 'No internet service' 'Yes']
StreamingMovies
['No' 'No internet service' 'Yes']
Contract
['Month-to-month' 'One year' 'Two year']
PaperlessBilling
['No' 'Yes']
PaymentMethod
['Bank transfer (automatic)' 'Credit card (automatic)' 'Electronic check'
 'Mailed check']


Look like there is no strange name.For example NO and no with only case differences.

We have to change the category feature into numeric feature since machine learning only work with numeric data

In [725]:
cat_columns = ["gender","SeniorCitizen","Partner","Dependents","PhoneService","MultipleLines",
           "InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport",
           "StreamingTV","StreamingMovies","Contract","PaperlessBilling",
           "PaymentMethod"]

cat_features = pd.get_dummies(features,columns=cat_columns)
cat_features.head()


,tenure,MonthlyCharges,TotalCharges,gender_Female,gender_Male,SeniorCitizen_0,SeniorCitizen_1,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,PhoneService_Yes,MultipleLines_No,MultipleLines_No phone service,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,1,29.85,29.85,1,0,1,0,0,1,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0
1,34,56.95,1889.5,0,1,1,0,1,0,1,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1
2,2,53.85,108.15,0,1,1,0,1,0,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1
3,45,42.30,1840.75,0,1,1,0,1,0,1,0,1,0,0,1,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,1,0,1,0,1,0,0,0
4,2,70.70,151.65,1,0,1,0,1,0,1,0,0,1,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0


Checking data type of features.
Column TotalCharges have the data type of object which can contain empty string value                           

In [726]:
cat_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 46 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   tenure                                   7043 non-null   int64  
 1   MonthlyCharges                           7043 non-null   float64
 2   TotalCharges                             7043 non-null   object 
 3   gender_Female                            7043 non-null   uint8  
 4   gender_Male                              7043 non-null   uint8  
 5   SeniorCitizen_0                          7043 non-null   uint8  
 6   SeniorCitizen_1                          7043 non-null   uint8  
 7   Partner_No                               7043 non-null   uint8  
 8   Partner_Yes                              7043 non-null   uint8  
 9   Dependents_No                            7043 non-null   uint8  
 10  Dependents_Yes                           7043 no

In [727]:

empty_index = cat_features[cat_features["TotalCharges"]==" "].index

print(f"Empty string index:{len(empty_index)}")


Empty string index:11


Since there are 11 empty string, we have to handle them somehow. I decided to remove the whole row which contain empty string

In [0]:
cat_features = cat_features.drop(index=empty_index)

label = label.drop(index=empty_index)

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

We need to split the data into training data and testing data. Training data will train the model and testing data will test how well the model is doing

In [0]:
X_train,X_test,y_train,y_test = train_test_split(cat_features,label,random_state=10,test_size=0.3)

In [731]:
lr_model = LogisticRegression(max_iter=len(cat_features))
lr_model.fit(X_train,y_train.values.reshape(-1))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=7032,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
from sklearn.metrics import classification_report

In [733]:
y_pred = lr_model.predict(X_test)

class_report = classification_report(y_test,y_pred)

print("Classification Report")
print(class_report)

Classification Report
              precision    recall  f1-score   support

          No       0.85      0.89      0.87      1551
         Yes       0.65      0.57      0.61       559

    accuracy                           0.80      2110
   macro avg       0.75      0.73      0.74      2110
weighted avg       0.80      0.80      0.80      2110



Based on Precision and Recall , we can see the recall rate of Yes is 0.57 which mean that only around 50% is correct.

What it mean that when the model said the customer will Churn (discontinue the service) , only around 57% is correct.

The main problem may be unbalance data (having one label type significantly  more than another label type)

We wanted to see how many whether our label is balance. If the label is unbalance we have to handle them in different way

In [0]:
new_data = pd.concat([X_train,y_train],axis=1)

In [735]:
def calculate_percentage(new_data:pd.DataFrame):
  total_label = len(new_data)

  yes_index = new_data[new_data["Churn"]=="Yes"].index

  no_index = new_data[new_data["Churn"]=="No"].index

  yes_data = new_data.loc[yes_index]

  no_data = new_data.loc[no_index]

  yes_length = len(yes_data)

  no_length = len(no_data)

  yes_percentage = (yes_length/total_label)*100

  no_percentage = (no_length/total_label)*100

  return yes_data,no_data,yes_length,no_length,yes_percentage,no_percentage

yes_data,no_data,yes_length,no_length,yes_percentage,no_percentage = calculate_percentage(new_data)

print(f"Yes label count:{yes_length}")
print(f"No label count:{no_length}")
print(f"Yes label percentage:{yes_percentage}")
print(f"No label percentage:{no_percentage}")


Yes label count:1310
No label count:3612
Yes label percentage:26.615197074360015
No label percentage:73.38480292563997


The Yes label type have around 26 and No label is type is around 73. Which is the text book case of unbalance data.

We need to balance the dataset. We can 


1.   Gather more data
2.   Undersample the label which have more data
3.   Oversample the label which have less data

We are going to undersample the label type which have No.

I wanted to have the equal number of No and Yes label

In [0]:
no_data_sample = no_data.sample(n=yes_length,random_state=123)

In [0]:
balance_data = pd.concat([yes_data,no_data_sample],axis=0)

In [738]:
yes_data,no_data,yes_length,no_length,yes_percentage,no_percentage = calculate_percentage(balance_data)

print(f"Yes label count:{yes_length}")
print(f"No label count:{no_length}")
print(f"Yes label percentage:{yes_percentage}")
print(f"No label percentage:{no_percentage}")


Yes label count:1310
No label count:1310
Yes label percentage:50.0
No label percentage:50.0


We can see that only new data set is balance with 50% of label type No and other 50% being label type Yes

In [739]:
balance_data.head()

,tenure,MonthlyCharges,TotalCharges,gender_Female,gender_Male,SeniorCitizen_0,SeniorCitizen_1,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,PhoneService_Yes,MultipleLines_No,MultipleLines_No phone service,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn
6538,2,79.85,152.45,0,1,1,0,1,0,1,0,0,1,1,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,Yes
3565,1,50.75,50.75,0,1,1,0,1,0,1,0,0,1,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,Yes
3721,2,20.65,38.7,1,0,1,0,1,0,1,0,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,1,Yes
39,11,97.85,1105.4,1,0,1,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,0,1,1,0,0,0,Yes
4941,3,19.80,58.15,0,1,1,0,1,0,1,0,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,Yes


Since we just append the row , we have to shuffle it

In [0]:
from sklearn.utils import shuffle

In [741]:
balance_data = shuffle(balance_data)
balance_data.head()

,tenure,MonthlyCharges,TotalCharges,gender_Female,gender_Male,SeniorCitizen_0,SeniorCitizen_1,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,PhoneService_Yes,MultipleLines_No,MultipleLines_No phone service,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn
5619,20,91.00,1859.5,1,0,0,1,0,1,1,0,0,1,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,1,1,0,0,0,No
1093,71,100.45,7159.7,1,0,1,0,0,1,0,1,0,1,0,0,1,0,1,0,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,0,1,0,No
5422,68,62.65,4375.8,0,1,1,0,0,1,0,1,0,1,1,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,No
6971,18,99.75,1836.25,1,0,0,1,0,1,1,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,1,0,0,0,Yes
851,52,85.15,4461.85,0,1,1,0,1,0,0,1,0,1,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1,No


We have to create a new testing data set based on our balance data set

In [742]:
X_train_new = balance_data.drop("Churn",axis=1)

y_test_new = balance_data[["Churn"]]

X_train_new.head()


,tenure,MonthlyCharges,TotalCharges,gender_Female,gender_Male,SeniorCitizen_0,SeniorCitizen_1,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,PhoneService_Yes,MultipleLines_No,MultipleLines_No phone service,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
5619,20,91.00,1859.5,1,0,0,1,0,1,1,0,0,1,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,1,1,0,0,0
1093,71,100.45,7159.7,1,0,1,0,0,1,0,1,0,1,0,0,1,0,1,0,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,0,1,0
5422,68,62.65,4375.8,0,1,1,0,0,1,0,1,0,1,1,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0
6971,18,99.75,1836.25,1,0,0,1,0,1,1,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,1,0,0,0
851,52,85.15,4461.85,0,1,1,0,1,0,0,1,0,1,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1


In [743]:
y_test_new.head()

,Churn
5619,No
1093,No
5422,No
6971,Yes
851,No


We create a new model with and train it with our balance data set

In [744]:
lr_model_new = LogisticRegression(max_iter=len(cat_features))
lr_model_new.fit(X_train_new,y_test_new.values.reshape(-1))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=7032,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [745]:
y_pred_new = lr_model_new.predict(X_test)

class_report = classification_report(y_test,y_pred_new)

print("Classification Report")
print(class_report)

Classification Report
              precision    recall  f1-score   support

          No       0.93      0.71      0.80      1551
         Yes       0.51      0.84      0.63       559

    accuracy                           0.74      2110
   macro avg       0.72      0.77      0.72      2110
weighted avg       0.81      0.74      0.76      2110



Let do some analysis.

First Model

                   precision recall  

          No       0.85      0.89      
         Yes       0.65      0.57      

Second Model

                   precision recall  

          No       0.93      0.71  
         Yes       0.51      0.84      


The first model is trained with unbalance data and the second model is train with balance data

We can see that the recall value for the Yes label type increase from 0.57% to 0.84% in second model.



Here are some info which you can use to decide the model performance

High recall + High precession = the class is perfectly handled by the model

Low recall + High precision = the model can’t detect the class well but is highly trustable when it does

High recall + Low precision = the class is well detected but the model also include points of other classes in it

Low recall + Low precision = the class is poorely handle by the model